# Loading Trained Model

In [4]:
import torch
from torch import nn

from src.Mymodel import MyResNet34
from src.Mymodel import MyResNet_CIFAR
from src.Mydataloader import LoadDataset
from src.Mytraining import DoTraining

In [5]:
"""Dataset selection"""
DATASET = "CIFAR10"
# DATASET = "CIFAR100"
# DATASET = "ImageNet2012"

"""Model selection for CIFAR"""
NUM_LAYERS_LEVEL = 5

"""Dataset parameters"""
BATCH = 128
SHUFFLE = True
NUMOFWORKERS = 8
PIN_MEMORY = True
SPLIT_RATIO = 0.9

"""optimizer parameters"""
OPTIMIZER = "SGD"
# OPTIMIZER = "Adam"
# OPTIMIZER = "Adam_decay"


file_path = ""
if DATASET == "ImageNet2012":
    file_path = f"MyResNet34_{BATCH}_{OPTIMIZER}"
    _model_name = f"MyResNet34_{DATASET}_{BATCH}_{OPTIMIZER}"

else:
    file_path = f"MyResNet{NUM_LAYERS_LEVEL*6+2}_{BATCH}_{OPTIMIZER}"
    _model_name = f"MyResNet{NUM_LAYERS_LEVEL*6+2}_{DATASET}_{BATCH}_{OPTIMIZER}"

if SPLIT_RATIO != 0:
    _model_name += f"_{int(SPLIT_RATIO*100)}"
    file_path += f"_{int(SPLIT_RATIO*100)}"

In [6]:
tmp = LoadDataset(root="data", seceted_dataset=DATASET, split_ratio=SPLIT_RATIO)
_, valid_data, test_data, COUNT_OF_CLASSES = tmp.Unpack()

train_dataloader, valid_dataloader, test_dataloader = tmp.get_dataloader(
    batch_size=BATCH, shuffle=SHUFFLE, num_workers=NUMOFWORKERS, pin_memory=PIN_MEMORY
)

"""Model selection"""
if DATASET == "CIFAR10":
    eval_dataloader = test_dataloader
elif DATASET == "ImageNet2012":
    eval_dataloader = valid_dataloader

print(len(eval_dataloader))

-----------------------------------------------------------------------
Dataset :  CIFAR10
- Length of Train Set :  45000
- Length of Valid Set :  5000
- Length of Test Set :  10000
- Count of Classes :  10
-----------------------------------------------------------------------
train.transforms = Compose(
      AutoAugment(interpolation=InterpolationMode.NEAREST, policy=AutoAugmentPolicy.CIFAR10)
      RandomCrop(size=(32, 32), padding=[4, 4, 4, 4], pad_if_needed=False, fill=0, padding_mode=constant)
      RandomHorizontalFlip(p=0.5)
      ToTensor()
      Normalize(mean=[0.49139968, 0.48215827, 0.44653124], std=[1, 1, 1], inplace=True)
) 128
valid.transforms = ToTensor() 128
test.transforms = ToTensor() 128
79


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if DATASET == "ImageNet2012":
    model = MyResNet34(num_classes=COUNT_OF_CLASSES, Downsample_option="B").to(device)

else:
    model = MyResNet_CIFAR(
        num_classes=COUNT_OF_CLASSES, num_layer_factor=NUM_LAYERS_LEVEL
    ).to(device)



model.load_state_dict(torch.load(f"models/{_model_name}/{file_path}.pth"))

<All keys matched successfully>

In [8]:
testingclass = DoTraining(
    model=model, criterion=nn.CrossEntropyLoss(), optimizer=None, device=device
)
test_loss, test_acc = testingclass.Forward_eval(eval_dataloader)

print("test_loss:", test_loss)
print(f"test_acc: {test_acc*100:.2f}%")
print(f"test_error: {100 - test_acc*100:.2f}%")
print(file_path)

test_loss: 0.5645339349402657
test_acc: 83.39%
test_error: 16.61%
MyResNet32_128_SGD_90
